In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Load the distraction CSV file
df_distraction = pd.read_csv('memory_match_distraction_data_200_samples.csv')

# Custom Dataset for sequence data
class DistractionDataset(Dataset):
    def __init__(self, dataframe, sequence_length=10):
        self.sequence_length = sequence_length
        self.features = dataframe[['reaction_time (sec)', 'off_task_event', 'eye_fixation_loss', 'is_match', 'move_number']].values.astype(np.float32)
        self.targets = dataframe['distraction_level'].values.astype(np.float32)

    def __len__(self):
        return len(self.features) - self.sequence_length

    def __getitem__(self, idx):
        x = self.features[idx:idx + self.sequence_length]
        y = self.targets[idx + self.sequence_length - 1]
        return torch.tensor(x), torch.tensor(y)

# GRU Model definition
class DistractionGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DistractionGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# Hyperparameters
INPUT_SIZE = 5  
HIDDEN_SIZE = 32
OUTPUT_SIZE = 1
EPOCHS = 20
BATCH_SIZE = 4
LEARNING_RATE = 0.001

# Prepare dataset and dataloader
dataset = DistractionDataset(df_distraction)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Initialize model, loss, optimizer
model = DistractionGRU(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(EPOCHS):
    total_loss = 0
    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(dataloader):.4f}")

# Example inference
with torch.no_grad():
    sample_seq = torch.tensor(dataset[0][0]).unsqueeze(0)  # batch size 1
    predicted_distraction = model(sample_seq)
    print(f"Predicted Distraction Level: {predicted_distraction.item():.4f}")

Epoch 1/20, Loss: 0.5934
Epoch 2/20, Loss: 0.0818
Epoch 3/20, Loss: 0.0387
Epoch 4/20, Loss: 0.0273
Epoch 5/20, Loss: 0.0222
Epoch 6/20, Loss: 0.0183
Epoch 7/20, Loss: 0.0144
Epoch 8/20, Loss: 0.0118
Epoch 9/20, Loss: 0.0109
Epoch 10/20, Loss: 0.0081
Epoch 11/20, Loss: 0.0070
Epoch 12/20, Loss: 0.0080
Epoch 13/20, Loss: 0.0056
Epoch 14/20, Loss: 0.0042
Epoch 15/20, Loss: 0.0039
Epoch 16/20, Loss: 0.0035
Epoch 17/20, Loss: 0.0027
Epoch 18/20, Loss: 0.0026
Epoch 19/20, Loss: 0.0021
Epoch 20/20, Loss: 0.0019
Predicted Distraction Level: 0.9873


C:\Users\KIIT01\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
